In [11]:
import pandas as pd
import os

def load_and_preprocess_data():
    """
    '../data/preprocessing' 폴더에서 모든 CSV 파일을 로드하고 '지역구' 기준으로 병합하는 함수.
    
    [중요] 원본 데이터 보호를 위해 이 폴더에서는 읽기만 수행합니다.
    이 코드는 '.../SORA_Project/src' 폴더 내의 노트북(test2.ipynb) 또는 스크립트에서 실행되는 것을 가정합니다.
    """
    print("--- 1. 데이터 로드 및 전처리 시작 ---")
    
    # 현재 스크립트(또는 노트북)의 CWD (예: .../SORA_Project/src)
    cwd = os.getcwd() 
    print(f"현재 작업 디렉토리(CWD): {cwd}")

    # [수정] CWD(.../src)에서 상위('..')로 이동 후 'data/preprocessing' 폴더로 접근
    data_folder = os.path.join(cwd, '..', 'data', 'preprocessing')

    print(f"데이터를 읽을 폴더 경로: {data_folder}")
    
    try:
        # 기준이 될 25개 자치구 리스트 로드 (X09...유흥주점.csv 사용)
        base_df = pd.read_csv(os.path.join(data_folder, 'X09_전처리_유흥주점_수.csv'), usecols=['지역구'])
        df_pubs = pd.read_csv(os.path.join(data_folder, 'X09_전처리_유흥주점_수.csv')).rename(columns={'지역구별 주점 수': 'Pubs'})

        # --- 1-1. 새로운 타겟(Y)을 위한 원본 데이터 로드 (from preprocessing) ---
        df_target_crime = pd.read_csv(os.path.join(data_folder, 'X03_전처리_범죄발생_건수_20231231기준.csv'), usecols=['지역구', '합계'])
        df_target_crime = df_target_crime.rename(columns={'합계': 'Sexual_Crime_Count'})

        df_risk_index = pd.read_csv(os.path.join(data_folder, 'X07_전처리_성폭력위험요소_지수.csv.csv'), usecols=['여성_잠재적_위험_지수'])
        df_risk_index = df_risk_index.rename(columns={'여성_잠재적_위험_지수': 'Sexual_Assault_Risk_Index'})
        df_risk_index['지역구'] = base_df['지역구'] # X07 파일에는 지역구 컬럼이 없음
        
        # --- 1-2. 안전 시설 변수(X) 로드 (from preprocessing) ---
        df_cctv = pd.read_csv(os.path.join(data_folder, 'Y06_전처리_서울시 안심귀갓길 안전시설물_CCTV.csv'))
        df_cctv = df_cctv.rename(columns={'CCTV 설치수': 'CCTV'})

        df_bell_raw = pd.read_csv(os.path.join(data_folder, 'Y06_전처리_안심벨_설치수_20230421기준.csv'))
        df_bell_agg = df_bell_raw.groupby('구')['안심벨 설치수'].sum().reset_index()
        df_bell_agg = df_bell_agg.rename(columns={'구': '지역구', '안심벨 설치수': 'Safety_Bell'})

        df_station = pd.read_csv(os.path.join(data_folder, 'Y14_전처리_지구대파출소_설치수_20241231기준.csv'))
        df_station = df_station.rename(columns={'지역구별 지구대파출소 수': 'Police_Station'})

        df_center = pd.read_csv(os.path.join(data_folder, 'Y14_전처리_치안센터_설치수_20250630기준.csv'))
        df_center = df_center.rename(columns={'지역구별 치안센터 수': 'Police_Center'})

        df_police_officers = pd.read_csv(os.path.join(data_folder, 'Y15_전처리_경찰관_인원수_2020_2024.csv'), usecols=['지역구', '2024'])
        df_police_officers = df_police_officers.rename(columns={'2024': 'Police_Officers'})

        # --- 1-3. 기타 참고 변수 로드 (from preprocessing) ---
        df_calls = pd.read_csv(os.path.join(data_folder, 'X02_전처리_112신고출동_연간건수_2020_2024.csv'), usecols=['지역구', '2023'])
        df_calls = df_calls.rename(columns={'2023': 'Calls_112'})

        df_elderly = pd.DataFrame(columns=['지역구', 'Elderly_Women_Alone']) # 빈 데이터프레임으로 초기화
        elderly_file_path = os.path.join(data_folder, 'X13_전처리_여성독거노인_현황_2020_2024.csv')
        if os.path.exists(elderly_file_path):
            df_elderly = pd.read_csv(elderly_file_path, usecols=['지역구', '2024'])
            df_elderly = df_elderly.rename(columns={'2024': 'Elderly_Women_Alone'})
        else:
            print("경고: 'preprocessing/X13_전처리_여성독거노인...csv' 파일을 찾을 수 없어 해당 데이터는 제외됩니다.")

        df_total_crime = pd.read_csv(os.path.join(data_folder, 'X01_전처리_5대범죄_발생검거_누적건수_2020_2024.csv'), usecols=['지역구', '총_누적_발생_건수'])
        df_total_crime = df_total_crime.rename(columns={'총_누적_발생_건수': 'Total_Crime_5'})


        # --- 모든 DataFrame 병합 ---
        df = base_df
        all_dfs = [
            df_target_crime, df_risk_index, # (Y 원본)
            df_cctv, df_bell_agg, df_station, df_center, df_police_officers, # (X)
            df_pubs, df_calls, df_elderly, df_total_crime # (기타)
        ]

        for temp_df in all_dfs:
            if not temp_df.empty:
                df = pd.merge(df, temp_df, on='지역구', how='left')

        df = df.fillna(0)
        df = df.set_index('지역구')
        
        if '남대문구' in df.index:
            df = df.drop('남대문구', errors='ignore') # '남대문구' 오타 수정
            
        print("--- 1. 데이터 로드 및 전처리 완료 ---")
        return df

    except FileNotFoundError as e:
        print(f"오류: '{e.filename}' 파일을 찾을 수 없습니다.")
        print(f"현재 CWD: {cwd}")
        print("이 노트북(test2.ipynb)이 '.../SORA_Project/src' 폴더 안에 있는지 확인하세요.")
        print(f"'{data_folder}' 경로 내에 모든 CSV 파일이 있는지 확인하세요.")
        return None
    except Exception as e:
        print(f"데이터 로드 중 오류 발생: {e}")
        return None

if __name__ == "__main__":
    
    # pandas 출력 옵션 설정 (터미널에서 전체를 보기 위함)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    
    main_df = load_and_preprocess_data()
    
    if main_df is not None:
        # [중요] 원본 보호를 위해 상위 폴더(이 경우 'SORA_Project' 루트)에 저장
        cwd = os.getcwd() # 예: .../SORA_Project/src
        output_path = os.path.join(cwd, '..', 'data_01_merged.pkl') # 예: .../SORA_Project/data_01_merged.pkl
        
        main_df.to_pickle(output_path)
        
        print(f"\n[저장 완료] 'data_01_merged.pkl' 파일이 상위 폴더({os.path.dirname(output_path)})에 생성되었습니다.")
        print("\n[병합된 데이터 상위 5개]")
        print(main_df.head())
        print("\n[데이터 정보 요약]")
        main_df.info()



--- 1. 데이터 로드 및 전처리 시작 ---
현재 작업 디렉토리(CWD): c:\Users\user\Desktop\경진대회\SORA_Project\src
데이터를 읽을 폴더 경로: c:\Users\user\Desktop\경진대회\SORA_Project\src\..\data\preprocessing
--- 1. 데이터 로드 및 전처리 완료 ---

[저장 완료] 'data_01_merged.pkl' 파일이 상위 폴더(c:\Users\user\Desktop\경진대회\SORA_Project\src\..)에 생성되었습니다.

[병합된 데이터 상위 5개]
     Sexual_Crime_Count  Sexual_Assault_Risk_Index  CCTV  Safety_Bell  지역구별 지구대/파출소 수  Police_Center  Police_Officers  Pubs  Calls_112  Elderly_Women_Alone  Total_Crime_5
지역구                                                                                                                                                                       
강남구                1099                       3902   122           89              14            2.0            931.0   165   120466.0              16359.0          33319
강동구                 294                       1103    48           46               9            0.0            737.0    98    91280.0              14763.0          17705
강북구  

In [13]:
!pip install scikit-learn

   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 8.7/8.7 MB 41.7 MB/s eta 0:00:00

   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- ------------- 2/3 [scikit-learn]
   ----

In [17]:
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

def create_target_variable():
    """
    1단계에서 저장된 'data_01_merged.pkl' 파일을 읽어와
    '통합 성범죄 위험도' (Y_target_new) 컬럼을 생성하고
    'data_02_with_target.pkl' 파일로 저장합니다.
    
    이 코드는 '.../data/src' 폴더 내의 노트북(test2.ipynb)에서 실행되는 것을 가정합니다.
    """
    print("--- 2. '통합 성범죄 위험도(Y)' 생성 시작 ---")
    
    # [수정] CWD가 '.../data/src'일 것을 가정하여 상위('..')에서 파일을 읽고 씁니다.
    cwd = os.getcwd()
    input_path = os.path.join(cwd, '..', 'data_01_merged.pkl')
    output_path = os.path.join(cwd, '..', 'data_02_with_target.pkl')
    
    try:
        if not os.path.exists(input_path):
            print(f"오류: '{input_path}' 파일을 찾을 수 없습니다.")
            print("먼저 01_load_data 스크립트(셀)를 실행하세요.")
            return None
            
        df = pd.read_pickle(input_path)
        
        # --- 1. '통합 성범죄 위험도' 생성 ---
        # 1-1. 정규화를 위한 변수 선택 (성범죄 건수, 성폭력 위험지수)
        target_vars = ['Sexual_Crime_Count', 'Sexual_Assault_Risk_Index']
        
        # 1-2. Min-Max 스케일러 (0~1 값으로 변환)
        scaler = MinMaxScaler()
        df[target_vars] = scaler.fit_transform(df[target_vars])
        
        # 1-3. 두 정규화 점수의 평균을 내어 '통합 성범죄 위험도' 생성
        df['Y_target_new'] = df[target_vars].mean(axis=1)
        
        df.to_pickle(output_path)
        
        print("--- 2. '통합 성범죄 위험도(Y)' 생성 완료 ---")
        print(f"\n[저장 완료] 'data_02_with_target.pkl' 파일이 상위 폴더({os.path.dirname(output_path)})에 생성되었습니다.")
        
        print("\n[Y_target_new 생성 확인 (상위 5개)]")
        print(df[['Sexual_Crime_Count', 'Sexual_Assault_Risk_Index', 'Y_target_new']].head())
        return df

    except Exception as e:
        print(f"2단계 실행 중 오류 발생: {e}")
        return None

if __name__ == "__main__":
    # pandas 출력 옵션 설정
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    
    create_target_variable()



--- 2. '통합 성범죄 위험도(Y)' 생성 시작 ---
--- 2. '통합 성범죄 위험도(Y)' 생성 완료 ---

[저장 완료] 'data_02_with_target.pkl' 파일이 상위 폴더(c:\Users\user\Desktop\경진대회\SORA_Project\src\..)에 생성되었습니다.

[Y_target_new 생성 확인 (상위 5개)]
     Sexual_Crime_Count  Sexual_Assault_Risk_Index  Y_target_new
지역구                                                             
강남구            1.000000                   1.000000      1.000000
강동구            0.121179                   0.159964      0.140572
강북구            0.085153                   0.189376      0.137264
강서구            0.407205                   0.342737      0.374971
관악구            0.494541                   0.555522      0.525032


In [18]:
import pandas as pd
import os
from scipy import stats
import json

def analyze_correlations():
    """
    2단계에서 저장된 'data_02_with_target.pkl' 파일을 읽어와
    'Y_target_new'와 안전시설(X) 변수 간의 상관관계를 분석합니다.
    유의미한(p < 0.05) 변수 정보는 'data_03_significant_factors.json'에 저장됩니다.
    
    이 코드는 '.../data/src' 폴더 내의 노트북(test2.ipynb)에서 실행되는 것을 가정합니다.
    """
    print("--- 3. 상관관계 분석 시작 ---")
    
    # [수정] CWD가 '.../data/src'일 것을 가정하여 상위('..')에서 파일을 읽고 씁니다.
    cwd = os.getcwd()
    input_path = os.path.join(cwd, '..', 'data_02_with_target.pkl')
    output_path = os.path.join(cwd, '..', 'data_03_significant_factors.json')
    
    try:
        if not os.path.exists(input_path):
            print(f"오류: '{input_path}' 파일을 찾을 수 없습니다.")
            print("먼저 02_create_target 스크립트(셀)를 실행하세요.")
            return None
            
        df = pd.read_pickle(input_path)
        
        target_var = 'Y_target_new'
        
        # 논문의 '안전시설'에 해당하는 변수 리스트
        safety_factors = ['CCTV', 'Safety_Bell', 'Police_Station', 'Police_Center', 'Police_Officers']
        
        correlations_data = []
        significant_factors_info = []
        
        print(f"\n'{target_var}'(통합 성범죄 위험도)와의 상관관계 분석 결과:")
        print("="*60)
        print(f"{'변수명 (X)':<20} | {'상관계수':<10} | {'p-value':<10} | {'유의미':<5}")
        print("-"*60)
        
        for factor in safety_factors:
            if factor not in df.columns:
                print(f"경고: '{factor}' 컬럼이 데이터에 없습니다. 건너뜁니다.")
                continue
            
            corr, p_value = stats.pearsonr(df[factor], df[target_var])
            is_significant = p_value < 0.05
            
            print(f"{factor:<20} | {corr:<10.3f} | {p_value:<10.3e} | {'*' if is_significant else ''}")
            
            correlations_data.append({
                "factor": factor,
                "correlation": corr,
                "p_value": p_value
            })
            
            # 논문 로직: 안전 시설 변수(safety_factors) 중에서 통계적으로 유의미한(p < 0.05) 것만 선정
            if is_significant:
                significant_factors_info.append({"factor": factor, "correlation": corr})

        print("="*60)
        print(f"* p-value < 0.05 (통계적으로 유의미한 변수)\n")
        
        # 유의미한 변수 정보 파일로 저장
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(significant_factors_info, f, indent=4, ensure_ascii=False)
            
        print("--- 3. 상관관계 분석 완료 ---")
        print(f"\n[저장 완료] 'data_03_significant_factors.json' 파일이 상위 폴더({os.path.dirname(output_path)})에 생성되었습니다.")
        print(f"\n[지수 산출에 사용될 유의미한 안전 시설]: {[f['factor'] for f in significant_factors_info]}")

        return significant_factors_info

    except Exception as e:
        print(f"3단계 실행 중 오류 발생: {e}")
        return None

if __name__ == "__main__":
    analyze_correlations()



--- 3. 상관관계 분석 시작 ---

'Y_target_new'(통합 성범죄 위험도)와의 상관관계 분석 결과:
변수명 (X)              | 상관계수       | p-value    | 유의미  
------------------------------------------------------------
CCTV                 | 0.568      | 3.073e-03  | *
Safety_Bell          | 0.518      | 7.996e-03  | *
경고: 'Police_Station' 컬럼이 데이터에 없습니다. 건너뜁니다.
Police_Center        | 0.027      | 8.986e-01  | 
Police_Officers      | 0.560      | 3.626e-03  | *
* p-value < 0.05 (통계적으로 유의미한 변수)

--- 3. 상관관계 분석 완료 ---

[저장 완료] 'data_03_significant_factors.json' 파일이 상위 폴더(c:\Users\user\Desktop\경진대회\SORA_Project\src\..)에 생성되었습니다.

[지수 산출에 사용될 유의미한 안전 시설]: ['CCTV', 'Safety_Bell', 'Police_Officers']


In [19]:
import pandas as pd
import os
import json

def calculate_safety_index():
    """
    'data_02_with_target.pkl' (데이터)와 'data_03_significant_factors.json' (유의미 변수)를 읽어
    논문 공식에 따라 '여성 안전시설 지수'를 산출합니다.
    결과는 'data_04_with_index.pkl' 파일로 저장됩니다.
    
    이 코드는 '.../data/src' 폴더 내의 노트북(test2.ipynb)에서 실행되는 것을 가정합니다.
    """
    print("--- 4. 안전시설 지수 산출 시작 ---")
    
    # [수정] CWD가 '.../data/src'일 것을 가정하여 상위('..')에서 파일을 읽고 씁니다.
    cwd = os.getcwd()
    data_path = os.path.join(cwd, '..', 'data_02_with_target.pkl')
    factors_path = os.path.join(cwd, '..', 'data_03_significant_factors.json')
    output_path = os.path.join(cwd, '..', 'data_04_with_index.pkl')
    
    try:
        if not (os.path.exists(data_path) and os.path.exists(factors_path)):
            print(f"오류: '{data_path}' 또는 '{factors_path}' 파일을 찾을 수 없습니다.")
            print("먼저 02, 03 스크립트(셀)를 실행하세요.")
            return None

        df = pd.read_pickle(data_path)
        with open(factors_path, 'r', encoding='utf-8') as f:
            significant_factors_info = json.load(f)
            
        if not significant_factors_info:
            print("오류: 지수 산출에 사용할 유의미한 변수가 없습니다.")
            print("03_correlate 스크립트(셀)의 결과를 확인하세요.")
            return None

        df['Total_Safety_Index'] = 0
        
        print("\n가중치 및 점수 계산:")
        print("="*60)
        print(f"{'변수명':<20} | {'상관계수':<10} | {'가중치 (1+Corr)':<15}")
        print("-"*60)
        
        rank_cols_to_show = []

        for item in significant_factors_info:
            factor = item['factor']
            corr = item['correlation']
            
            # 가중치 = 1 + 상관계수 (논문 변형 공식)
            weight = 1 + corr
            print(f"{factor:<20} | {corr:<10.3f} | {weight:<15.3f}")

            # 논문 공식: 설치 수가 많을수록 높은 점수 부여 (1점 ~ 25점)
            score_col = f'{factor}_Score'
            # 'min' 방식: 동점일 경우 가장 낮은 등수 부여 (예: 1, 2, 2, 4)
            df[score_col] = df[factor].rank(method='min', ascending=True) 
            
            # 최종 지수 = (시설A 점수 * 시설A 가중치) + (시설B 점수 * 시설B 가중치) ...
            df['Total_Safety_Index'] += df[score_col] * weight
            
            rank_cols_to_show.extend([factor, score_col])
        
        print("="*60)
        
        df.to_pickle(output_path)

        print("--- 4. 안전시설 지수 산출 완료 ---")
        print(f"\n[저장 완료] 'data_04_with_index.pkl' 파일이 상위 폴더({os.path.dirname(output_path)})에 생성되었습니다.")

        print("\n[시설별 설치 수 및 순위 점수 (상위 5개)]")
        if rank_cols_to_show:
            print(df[rank_cols_to_show].sort_values(by=rank_cols_to_show[0], ascending=False).head())
        
        print("\n[최종 지수 (스케일링 전) (상위 5개)]")
        print(df[['Total_Safety_Index']].sort_values(by='Total_Safety_Index', ascending=False).head())
        
        return df

    except Exception as e:
        print(f"4단계 실행 중 오류 발생: {e}")
        return None

if __name__ == "__main__":
    # pandas 출력 옵션 설정
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    
    calculate_safety_index()



--- 4. 안전시설 지수 산출 시작 ---

가중치 및 점수 계산:
변수명                  | 상관계수       | 가중치 (1+Corr)   
------------------------------------------------------------
CCTV                 | 0.568      | 1.568          
Safety_Bell          | 0.518      | 1.518          
Police_Officers      | 0.560      | 1.560          
--- 4. 안전시설 지수 산출 완료 ---

[저장 완료] 'data_04_with_index.pkl' 파일이 상위 폴더(c:\Users\user\Desktop\경진대회\SORA_Project\src\..)에 생성되었습니다.

[시설별 설치 수 및 순위 점수 (상위 5개)]
     CCTV  CCTV_Score  Safety_Bell  Safety_Bell_Score  Police_Officers  Police_Officers_Score
지역구                                                                                          
관악구   158        25.0           49               21.0            894.0                   20.0
강남구   122        24.0           89               25.0            931.0                   22.0
강북구    92        23.0           77               24.0            652.0                   10.0
성북구    86        22.0           71               23.0            47

In [20]:
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

def show_final_results():
    """
    4단계에서 저장된 'data_04_with_index.pkl' 파일을 읽어
    최종 지수를 0~100점으로 스케일링하고, 순위를 터미널에 출력합니다.
    
    이 코드는 '.../data/src' 폴더 내의 노트북(test2.ipynb)에서 실행되는 것을 가정합니다.
    """
    print("--- 5. 최종 결과 확인 시작 ---")
    
    # [수정] CWD가 '.../data/src'일 것을 가정하여 상위('..')에서 파일을 읽습니다.
    cwd = os.getcwd()
    input_path = os.path.join(cwd, '..', 'data_04_with_index.pkl')
    
    try:
        if not os.path.exists(input_path):
            print(f"오류: '{input_path}' 파일을 찾을 수 없습니다.")
            print("먼저 04_calculate_index 스크립트(셀)를 실행하세요.")
            return None
            
        df = pd.read_pickle(input_path)

        # 최종 지수를 0~100점 척도로 스케일링하여 가독성 확보
        scaler = MinMaxScaler(feature_range=(0, 100))
        df['Safety_Index_Scaled'] = scaler.fit_transform(df[['Total_Safety_Index']])
        
        # 순위 컬럼 추가 (1위 ~ 25위)
        df['Rank'] = df['Safety_Index_Scaled'].rank(method='min', ascending=False).astype(int)
        
        # 최종 결과 정렬 및 출력
        final_cols = ['Rank', 'Safety_Index_Scaled', 'Total_Safety_Index']
        # 사용된 변수들을 동적으로 추가 (Score 컬럼 제외)
        score_cols = [col for col in df.columns if col.endswith('_Score')]
        factors_used = [col.replace('_Score', '') for col in score_cols]
        
        final_cols.extend(factors_used)
        
        df_final_result = df[final_cols].sort_values(by='Rank', ascending=True)

        print("\n--- [최종] 서울시 지역구별 여성 안전시설 지수 (0~100점) ---")
        print(df_final_result)
        
        return df_final_result

    except Exception as e:
        print(f"5단계 실행 중 오류 발생: {e}")
        return None

if __name__ == "__main__":
    # pandas 출력 옵션 설정
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    
    show_final_results()



--- 5. 최종 결과 확인 시작 ---

--- [최종] 서울시 지역구별 여성 안전시설 지수 (0~100점) ---
      Rank  Safety_Index_Scaled  Total_Safety_Index  CCTV  Safety_Bell  Police_Officers
지역구                                                                                    
강남구      1           100.000000          109.888227   122           89            931.0
관악구      2            91.781385          102.264752   158           49            894.0
송파구      3            81.757849           92.967057    65           47           1057.0
강북구      4            76.496329           88.086543    92           77            652.0
서초구      5            73.385665           85.201134    74           47            779.0
영등포구     6            66.769908           79.064448    58           41            989.0
동작구      7            64.781351           77.219889    69           58            644.0
노원구      8            63.343377           75.886045    60           46            781.0
용산구      9            61.904110           74.551001   